Created on Monday 04 January 2021  

**Group 2 - Recherche de nouvelles sources**  
**Scraping de site**

@authors : Maël Lesavourey, Sibel Yüksel, Rémy Lapeyre

# Librairies

In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from tqdm import tqdm
from datetime import datetime
from datetime import date

# Query on 20minutes

In [93]:
url_site_1 : str = 'https://www.20minutes.fr/search?q='
url_site_2 : str = 'https://www.journaldunet.com/recherche/?f_libelle='
url_site_3 : str = 'https://www.lemondeinformatique.fr/recherche/index.html?search='
url_site_4 : str = 'https://www.usine-digitale.fr/recherche='
url_site_5 : str = 'https://www.usinenouvelle.com/recherche='
url_site_6 : str = 'https://www.lemonde.fr/recherche/?search_keywords='

In [94]:
path = 'C:/Users/AAA/Documents/Cours/L3/Projet Interpromo/Chromedriver/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(url_site_5)

In [158]:
def get_query_src_page(pages: int, keywords: list, source_url: str, search_url: str,
                       url_attr: str = None, separator: str = '+') -> list:
    """Documentation

    Get the source code of the pages resulting a research of a keyword.

    Parameters:
        pages: The number of pages you want to retrieve.
        keywords: List of keywords used in the research.
        source_url: URL of the website.
        search_url: The part of URL before keywords (ex: search?q=).
        url_attr: PHP attributes that may be after the keyword.

    Out:
        page_list: List containing the source code of each page of the research.
    """
    # Initialize webdriver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    wd = webdriver.Chrome(path, options=chrome_options)

    # Base url
    base = source_url + search_url
    if url_attr is None:
        for kw in keywords:
            base += kw + separator
        base = base[:-len(separator)]
    else:
        for kw in keywords:
            base += kw + separator
        base = base[:-len(separator)]
        base += '/' + url_attr
    page_list = []
    print(base)
    for i in tqdm(range(pages)):
        # Simulate a webdriver instance and get the source page 
        k = i + 1
        wd.get(base + '&page=' + str(k))
        # Transform it with the BS' html parser
        soup = BeautifulSoup(wd.page_source)
        page_list.append(soup)
        """response = requests.get(base)
        soup = BeautifulSoup(response.text, 'html.parser')
        page_list.append(soup)"""
    return page_list

In [164]:
#pages = get_query_src_page(pages = 1, keywords = ['ressources','humaines'], source_url = 'https://www.cnil.fr', search_url = '/fr/recherche/', separator = '%20')
#pages = get_query_src_page(pages = 1, keywords = ['innovation'], source_url = 'https://www.fnccr.asso.fr/', search_url = '?s=')
pages = get_query_src_page(pages = 1, keywords = ['ressources','humaines'], source_url = 'https://www.riskinsight-wavestone.com/', search_url = '?s=')

pages

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

https://www.riskinsight-wavestone.com/?s=ressources+humaines


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.36s/it]


[<html class="js cssanimations csstransitions svg inlinesvg svgclippaths" prefix="og: http://ogp.me/ns#"><!--<![endif]--><head>
 <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
 <meta content="https://www.riskinsight-wavestone.com/wp-content/uploads/2014/11/fotolia_65227694-600x510.jpg" property="og:image"/>
 <meta content="https://www.riskinsight-wavestone.com/wp-content/uploads/2014/11/fotolia_65227694-600x510.jpg" name="twitter:image"/>
 <link href="https://www.riskinsight-wavestone.com/wp-content/themes/solucom-insight/images/favicon.png" rel="icon" type="image/png"/>
 <!--[if IE]><link rel="shortcut icon" type="image/x-icon" href="https://www.riskinsight-wavestone.com/wp-content/themes/solucom-insight/images/favicon.ico" /><![endif]-->
 <title>Vous avez cherché ressources humaines - RiskInsight</title>
 <link href="https://www.riskinsight-wavestone.com/wp-content/themes/solucom

In [160]:
def get_art_url(page_list: list, source_url: str, tag: str, attr: dict = None) -> list:
    """Documentation:

    Get the article's urls.
   
    Parameters:
        page_list: list of pages' source code.
        source_url: URL of source website.
        tag:  tag of the closest identifiable tag.
        attr: attribute of the closest identifiable tag.
        
    Out:
        url_list: list of the urls.
    """

    for page in tqdm(page_list):
        tag_list = page.find_all(tag, attr)

    url_list = []

    """for article in tag_list:  
        hyperlink_list = article.find_all('a') # <a> tags contain the urls
        print(hyperlink_list)
        for hyperlink in hyperlink_list:
            # Add the attribute 'href' which contains the url of the article
            url_list.append(hyperlink['href'])"""
    if tag is 'a':
        for article in tag_list:
            url_list.append(article['href'])
    else :
        for article in tag_list:
            hyperlink = article.find('a') # <a> tags contain the urls
            url_list.append(hyperlink['href'])
        # Delete potential duplicates
    url_list = list(set(url_list))
    
    # Delete specific urls
    url_list_clean: list = []
    for i in range(len(url_list)):
        if ('http' not in url_list[i]):
            if (url_list[i] != '#'):
                url_list_clean.append(source_url + url_list[i])
        else :
            if source_url in url_list[i]:
                url_list_clean.append(url_list[i])
    return url_list_clean

<>:27: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:27: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-160-9dee75b801ea>:27: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tag is 'a':


In [165]:
#get_art_url(pages, "div",{"class":"gsc-webResult gsc-result"})     #20minutes et Journaldunet
#get_art_url(pages, "article", {"class":"item"})     #LeMondeInformatique
#get_art_url(pages, "div", {"class":"blocResultRech"})     #Usine-digitale
#get_art_url(pages, "section", {"class":"blocType1"})      #UsineNouvelle
#len(get_art_url(pages, "section", {"class":"teaser teaser--inline-picture"})) #LeMonde
test = get_art_url(page_list = pages, source_url = "https://www.riskinsight-wavestone.com/", tag = "article", attr = {"class" : "post-resume"})
print(test)
print(len(test))

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 166.71it/s]

['https://www.riskinsight-wavestone.com/2014/11/gestion-portefeuille-projets-lautre-levier-damelioration-competitivite/', 'https://www.riskinsight-wavestone.com/2014/01/rssi-communiquez-avec-ceux-qui-communiquent/', 'https://www.riskinsight-wavestone.com/2013/01/la-fabric-paas-coeur-de-la-proposition-de-valeur-du-paas-prive/', 'https://www.riskinsight-wavestone.com/2013/04/developpement-international-quelle-protection-sociale-pour-les-salaries-de-nouvelles-filiales/', 'https://www.riskinsight-wavestone.com/2014/05/ressources-humaines-big-data-quel-avenir/', 'https://www.riskinsight-wavestone.com/2017/03/evolution-de-la-gestion-des-ressources-humaines-quels-impacts-sur-la-protection-des-donnees-personnelles/', 'https://www.riskinsight-wavestone.com/2016/02/choc-extreme-comment-se-preparer-a-une-crue-centennale-en-ile-de-france-22/', 'https://www.riskinsight-wavestone.com/2012/09/continuite-dactivite-noubliez-pas-dimpliquer-les-ressources-humaines/', 'https://www.riskinsight-wavestone.co

In [99]:
def get_datetime(str_date):
    """Documentation
    
    Parameters:
        str_date: date written for human comprehension ("6 février 2020")
    
    Out:
        date_format_datetime: datetime corresponding to the date
    """
    trans_month = {'01':['janv.','janvier'], 
             '02':['févr.','février'],
             '03':['mars'],
             '04':['avr.','avril'],
             '05':['mai'],
             '06':['juin'],
             '07':['juil.','juillet'],
             '08':['août'],
             '09':['sept.','septembre'],
             '10':['oct.','octobre'],
             '11':['nov.','novembre'],
             '12':['déc.','décembre']}
    date_tab = str_date.split(" ")
    day = date_tab[0]
    month = date_tab[1]
    for m in trans_month:
        if month.lower() in trans_month[m]:
            month = m
    year = date_tab[2]
    date_format_datetime = date(int(year), int(month), int(day))
    return date_format_datetime

In [100]:
month_list = ['janv.','janvier', 
             'févr.','février',
             'mars',
             'avr.','avril',
             'mai',
             'juin',
             'juil.','juillet',
              'août',
             'sept.','septembre',
             'oct.','octobre',
             'nov.','novembre',
             'déc.','décembre']

In [175]:
def scraping_research(page_list, source_url, html_tag_txt, html_attribute_txt, html_tag_articles, html_attribute_articles, date_format_code):
    url_list = get_art_url(page_list, source_url, html_tag_articles, html_attribute_articles)
    optimised_url_list = list()
    cpt_all_articles = 0
    for page in page_list:   
        html_list = []
        html_list_articles = page.find_all(html_tag_articles, html_attribute_articles)
        #print(html_list_articles)
    
        for cpt in range(0,len(html_list_articles)):
            iteration_date_article = html_list_articles[cpt].find(html_tag_txt, html_attribute_txt)
            html_list.append(iteration_date_article)
        if  html_tag_txt == "div" and html_attribute_txt == {"class":"gs-bidi-start-align gs-snippet"}:
            html_list = html_list[0:len(html_tag_txt)-1]
        #print(html_list)
        #date_list = list()
        for article in html_list:
            #Retrieve the dates
            if article is None:
                optimised_url_list.append(url_list[cpt_all_articles])
                cpt_all_articles += 1
                continue
            if source_url == 'https://citoyen-ne-s-de-marseille.fr/':
                date_article = article['content']
            else:
                date_article = article.get_text().replace('\n','').replace('\xa0','')
            print(date_article)
            try :
                date_article = datetime.strptime(date_article, date_format_code).date()
            except :    
                if 'Mis à jour le' in date_article and html_attribute_txt == {"class":"gs-bidi-start-align gs-snippet"}:
                    date_article = date_article.split('Mis à jour le ')[1].split()[0]
                    date_article = datetime.strptime(date_article,'%d/%m/%y').date()
                    
                else:
                    if 'Publié le' in date_article:
                        date_article = date_article.split('Publié le ')[0]
                    date_article = date_article.split()
                    #print(date_article)
                    if len(date_article[0]) in [1,2] and date_article[1] in month_list and len(date_article[2]) == 4:
                        date_article = get_datetime(date_article[0] + ' ' + date_article[1] + ' ' + date_article[2])
                    else:
                        date_article = date(2020, 2, 27)
            #date_list.append(date_article)
            #print(cpt_all_articles)
            # Retrieve dates that are >= 2020
            #print(type(date(2019, 12, 31)))
            if date_article > date(2019, 12, 31):
                optimised_url_list.append(url_list[cpt_all_articles])
            cpt_all_articles += 1
    #print(len(optimised_url_list))
    return optimised_url_list

#print(scraping_research(pages,"div",{"class":"gs-bidi-start-align gs-snippet"},"div",{"class":"gsc-webResult gsc-result"},"fr")) #20minutes et Journaldunet
#print(scraping_research(pages, "p", {"class":"dateRech"}, "div", {"class":"blocResultRech"},"%d/%m/%Y")) #Usine-Digitale
#print(scraping_research(pages, "span", {"class":"dateArt"}, "section", {"class":"blocType1"}, "%d/%m/%Y - %Hh%M")) #UsineNouvelle
test = scraping_research(pages, 'https://www.riskinsight-wavestone.com/', "p", {"class":"post-resume--date"} , "article", {"class" : "post-resume"}, "%d/%m/%Y")
print(test)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.05it/s]

Publié le  23/05/2014


IndexError: list index out of range